In [11]:
import numpy as np
import pandas as pd
import math
import itertools
import sys
import torch  
import numpy as np  
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm 
from scipy.stats import beta

import statistics
import json

import import_ipynb
from Environments import Square_Crossroads
# from Actor_Critic import Actor

In [12]:
class Actor(nn.Module):
    def __init__(self, num_inputs, hidden_size_actor):
        super(Actor, self).__init__()
      
        self.num_inputs = num_inputs
        self.actor_linear1 = nn.Linear(self.num_inputs, hidden_size_actor)
        self.actor_linear2 = nn.Linear(hidden_size_actor, hidden_size_actor)
        self.alpha = nn.Linear(hidden_size_actor, 1)
        self.beta = nn.Linear(hidden_size_actor, 1)
        
        self.pos_neg_1 = nn.Linear(self.num_inputs, 10)
        self.pos_neg_2 = nn.Linear(10, 2)

    def forward(self, state_tensor):

        policy_dist = F.relu(self.actor_linear1(state_tensor))
        policy_dist = F.relu(self.actor_linear2(policy_dist))

        policy_dist_alpha = torch.clamp(self.alpha(policy_dist), 0.1, 3)
        policy_dist_beta = torch.clamp(self.beta(policy_dist), 0.1, 3)
        
        sign = F.relu(self.pos_neg_1(state_tensor))
        sign = F.softmax(self.pos_neg_2(sign))
        
        return policy_dist_alpha, policy_dist_beta, sign
    
class Critic(nn.Module):
    def __init__(self, num_inputs, hidden_size_critic):
        super(Critic, self).__init__()
      
        self.num_inputs = num_inputs
        
        
        self.critic_linear1 = nn.Linear(self.num_inputs, hidden_size_critic)
        self.critic_linear2 = nn.Linear(hidden_size_critic, 1)


    def forward(self, state_tensor):
        state_tensor = Variable(state_tensor.float())
        value = F.relu(self.critic_linear1(state_tensor))
        value = self.critic_linear2(value)
        
        
        return value

In [13]:
n_times_steps = 3
seed = 10
dist_cars = 0.1

env = Square_Crossroads(seed, dist_cars)
env.reset()
print(env.states)
# action_space = {0:[0,0], 1:[0,0], 2:[0,0], 3:[0,0]}
# total_reward = 0

# state_tensor = torch.zeros([4,2], dtype=torch.float32)


# ac = Actor_Critic(state_tensor, 20, 400)

Environment reset with param
Environment reset with param
{0: [[0, 5], [5, 10], 1, 1], 1: [[5, 0], [0, 5], 1, 1], 2: [[5, 10], [5, 0], 1, 1], 3: [[10, 5], [5, 0], 1, 1]}


In [14]:
env.states

{0: [[0, 5], [5, 10], 1, 1],
 1: [[5, 0], [0, 5], 1, 1],
 2: [[5, 10], [5, 0], 1, 1],
 3: [[10, 5], [5, 0], 1, 1]}

In [15]:
def train_Q_actor_critic(env, hyperparameters):
    
    model = Actor_Critic(2, hyperparameters["hidden_size_actor"], hyperparameters["hidden_size_critic"])
    model_optimizer = optim.Adam(model.parameters(), lr = hyperparameters["lr"])
    model_scheduler = optim.lr_scheduler.StepLR(model_optimizer, step_size = hyperparameters["step_size"], gamma = hyperparameters["gamma"])
    
    all_lengths = []
    average_lengths = []
    all_rewards = []
    
    l_actor = []
    l_critic = []
    l_total = []
    all_reward = []
    for episode in range(max_episodes):
        print("Training episode: ", episode)
        
        log_probs = []
        values = []
        rewards = []
        
        env.reset()
        
        states = [v[0] for v in env.states.values()] 
        state_space = torch.tensor([[states[0][0], states[0][1]], 
                                    [states[1][0], states[1][1]],
                                    [states[2][0], states[2][1]],
                                    [states[3][0], states[3][1]]], dtype=torch.float32)

        for steps in tqdm(range(num_steps)):
            
            log_prob = 0
            actions = []
            
            value, policy_dist_alpha, policy_dist_beta, sign = model.forward(state_space)
            
            
                
            value = value.detach().numpy().mean()
#             print(value)
            for n in range(4):
                tmp_alpha = policy_dist_alpha[n].clone()
                tmp_beta = policy_dist_beta[n].clone()
                
                dist = beta(tmp_alpha.detach().numpy(),tmp_beta.detach().numpy())
                
                action = np.random.beta(tmp_alpha.detach().numpy(), tmp_beta.detach().numpy(), size=2)
                log_prob += torch.mean(torch.tensor(dist.logpdf(action)))
                

                if torch.argmax(sign[n]) == 1:
                    action[0] = -action[0]

                actions.append(action)
   
            action_space = torch.tensor(actions, dtype=torch.float32)                
            
            new_state_space, reward, done = env.step(action_space, sign)
            
            
            advantage = reward + (1-done)*gamma*critic(t(next_state)) - critic(t(state))
            
            rewards.append(reward)
            values.append(value)
            log_probs.append(log_prob.mean())
            
            states = [v[0] for v in env.states.values()] 

            new_state_space = torch.tensor([[states[0][0], states[0][1]], 
                                            [states[1][0], states[1][1]],
                                            [states[2][0], states[2][1]],
                                            [states[3][0], states[3][1]]], dtype=torch.float32)


            state_space = new_state_space

            if done or steps == num_steps-1:
                Qval, _, _, _ = model.forward(state_space)
#                 print("Qval: ",Qval)
                Qval = torch.mean(Qval).detach().numpy()
#                 print("Qval averaged: ",Qval)
                
                Qval = Variable(torch.FloatTensor(Qval), requires_grad = True)
#                 print(Qval.requires_grad)
                
                all_rewards.append(np.sum(rewards))
                all_lengths.append(steps)
                average_lengths.append(np.mean(all_lengths[-10:]))
#                 if episode % 10 == 0:                    
                sys.stdout.write("episode: {}, reward: {}, total length: {}, average length: {} \n".format(episode, np.sum(rewards), steps, average_lengths[-1]))
                print("Episode finished")
                break
        
        # compute Q values
        Qvals = np.zeros_like(values)
        for t in reversed(range(len(rewards))):

            Qval = rewards[t] + GAMMA * Qval
            Qvals[t] = Qval
        
        #update actor critic
        

        # suppose everything have the correct type
        # the term 'done' is important because for the end of the episode we only want
        # the reward, without the discounted next state value.
        advantage = reward + (1.0 - done) * gamma * critic(next_state) - critic(state)

        values = Variable(torch.FloatTensor(values), requires_grad=True)
        Qvals = Variable(torch.FloatTensor(Qvals), requires_grad=True)
        
        log_probs = torch.stack(log_probs)
        
        advantage = Qvals - values
        
        avg_advantage = advantage.mean()
        avg_log_probs = log_probs.mean()
        
        loss_actor = (avg_log_probs * advantage).mean()
        
        loss_critic = huber_loss(avg_advantage, delta)
        
#         loss_critic = abs(0.5*advantage.mean())
        
#         loss_actor = (-(policy_dist_alpha.sum() + policy_dist_beta.sum() + 1e-5)) * advantage   
        
        print("cricitc loss is: .........",loss_critic)
        print("actor loss is: .........", loss_actor)
        total_loss = loss_actor + loss_critic
        
        print("The total loss is: ",total_loss)
        
        l_actor.append(loss_actor)
        l_critic.append(loss_critic)
        l_total.append(total_loss)
        
        model_optimizer.zero_grad()
        total_loss.backward()
        model_optimizer.step()
#         model_scheduler.step()
        
        all_reward.append(statistics.mean(rewards))

    plt.plot(l_actor)
    plt.legend("Actor loss")    
    plt.show()

    plt.plot(l_critic)
    plt.legend("Critic loss")    
    plt.show()
    
    plt.plot(l_total)
    plt.legend("Total loss")    
    plt.show()

    plt.plot(all_reward)
    plt.legend("Mean reward")    
    plt.show()

In [16]:
hyperparameters = {"hidden_size_actor" : 200,
                   "hidden_size_critic" : 400,
                   "lr_actor" : 1e-4,
                   "lr_critic" : 1e-4,
                   "step_size" : 5,
                   "gamma" : 0.9}

max_episodes = 3
GAMMA = 0.99
delta = 1

In [17]:
def huber_loss(advantage, delta = 1.0):
    
    '''
    Huber loss, given advantage
    
    '''
    
    error = abs(advantage).sum()
    
    if error.item() > delta:
        loss = 0.5 * delta ** 2 + delta * (error - delta)

    else: 
        loss = 0.5 * error ** 2

    return loss

In [120]:
def train_A2C(env, hyperparameters):
    
    actor = Actor(2, hyperparameters["hidden_size_actor"])
    actor_optimizer = optim.Adam(actor.parameters(), lr = hyperparameters["lr_actor"])
    actor_scheduler = optim.lr_scheduler.StepLR(actor_optimizer, step_size = hyperparameters["step_size"], gamma = hyperparameters["gamma"])
    
    critic = Critic(2, hyperparameters["hidden_size_critic"])
    critic_optimizer = optim.Adam(critic.parameters(), lr = hyperparameters["lr_critic"])
    critic_scheduler = optim.lr_scheduler.StepLR(critic_optimizer, step_size = hyperparameters["step_size"], gamma = hyperparameters["gamma"])
    
    
    all_lengths = []
    average_lengths = []
    all_rewards = []
    
    l_actor = []
    l_critic = []
    l_total = []
    all_reward = []
    
    states_unity = {}
    
    episode_rewards = []
    for episode in range(max_episodes):
        
        states_episode = {}

        total_reward = 0
        steps = 0
        done = False

        print("Training episode: ", episode)

        
        env.reset()
        
        states = [v[0] for v in env.states.values()] 
        state_space = torch.tensor([[states[0][0], states[0][1]], 
                                    [states[1][0], states[1][1]],
                                    [states[2][0], states[2][1]],
                                    [states[3][0], states[3][1]]], dtype=torch.float32)
        
        states_episode[str(steps)] = {'0': [1, 1], '1': [1, 1], '2': [1, 1], '3': [1, 1]}
        
        
        while not done:
            steps+=1

                
            log_probs = []
            actions = []
            
            policy_dist_alpha, policy_dist_beta, sign = actor.forward(state_space)
            
            value_current = critic.forward(state_space)
            for n in range(4):
                tmp_alpha = policy_dist_alpha[n].clone()
                tmp_beta = policy_dist_beta[n].clone()
                
                dist = beta(tmp_alpha.detach().numpy(),tmp_beta.detach().numpy())
                
                action = np.random.beta(tmp_alpha.detach().numpy(), tmp_beta.detach().numpy(), size=2)
                log_prob = torch.mean(torch.tensor(dist.logpdf(action)))
                log_probs.append(log_prob)
                
                p = sign[n].clone().detach().numpy()
                action_sign = np.random.choice((1,-1), p=p)
                action[0] = action[0] * action_sign
                
                actions.append(action)
            
            action_space = torch.tensor(actions, dtype=torch.float32)                
            
            new_state_space, reward, done = env.step(action_space, sign)
            
            new_state_space = {str(k):v for k, v in new_state_space.items()}
            
            modified_dict = {}
            for k, v in new_state_space.items():
                tmp_v = []

                
                if torch.is_tensor(v[2]):
                    tmp_v.append(float(v[2].numpy()))
                
                if torch.is_tensor(v[3]):
                    tmp_v.append(float(v[3].numpy()))
                            
                modified_dict[k] = tmp_v
                
            states_episode[str(steps)] = modified_dict
            
            states = [v[0] for v in env.states.values()] 
            new_state_space = torch.tensor([[states[0][0], states[0][1]], 
                                            [states[1][0], states[1][1]],
                                            [states[2][0], states[2][1]],
                                            [states[3][0], states[3][1]]], dtype=torch.float32)

            state_space = new_state_space
            
            value_next = critic.forward(state_space)
            
            advantage = (reward + (1-done)*GAMMA*value_next - value_current).squeeze()
            
            total_reward += reward
            
            

            tmp = torch.zeros([4, 1], dtype=torch.float32)
            log_probs = torch.stack(log_probs, out=tmp)
            log_probs = Variable(log_probs, requires_grad=True)

#             critic_loss = (advantage**2).mean()
            critic_loss = huber_loss(advantage, delta)
            
            actor_loss = torch.mean(-log_probs*advantage.detach())
            
            total_loss = critic_loss+actor_loss

            actor_optimizer.zero_grad()
            actor_loss.backward()
            actor_optimizer.step()
            
            critic_optimizer.zero_grad()
            critic_loss.backward()
            critic_optimizer.step()
            
            l_actor.append(actor_loss)
            l_critic.append(critic_loss)
            l_total.append(total_loss)
            
            if steps%250 == 0: 
                print (steps)

            if done:
                print("The total episodic characteristics are as follows:")
                print("Critic loss: ",critic_loss)
                print("Actor loss: ",actor_loss)
                print("Total loss: ",total_loss)
                
                states_unity[episode] = states_episode

                with open(f'episode_{episode}.txt', 'w') as file:
                     file.write(json.dumps(states_episode))
                
        episode_rewards.append(total_reward)
        
    return episode_rewards, states_unity

## Training

In [126]:
episode_rewards,states_unity = train_A2C(env, hyperparameters)

In [ ]:

plt.scatter(np.arange(len(episode_rewards)), episode_rewards, s=2)
plt.title("Total reward per episode (online)")
plt.ylabel("reward")
plt.xlabel("episode")
plt.show()

In [ ]:
# data
df=pd.DataFrame({'x': range(1,501), 'y': episode_rewards })
plt.rcParams["figure.figsize"] = (20,20)
plt.plot(df['y'].rolling(5).mean(),label='data')
plt.show()
